In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
path = 'dataset/'

In [3]:
# processing the timeline
def dataPreProcessTime(df):
    #df['click_time'] = pd.to_datetime(df['click_time']).dt.date
    #df['click_time'] = df['click_time'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
    df['month'] = pd.to_datetime(df['click_time']).dt.month.astype(int)
    df['hour'] = pd.to_datetime(df['click_time']).dt.hour.astype(int)
    df['day'] = pd.to_datetime(df['click_time']).dt.day.astype(int)
    return df

In [4]:
start_time = time.time()

In [5]:
train = pd.read_csv(path+"train.csv", skiprows=0, nrows=10000000)
test = pd.read_csv(path+"test.csv")
train.columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time', 'is_attributed']

print('[{}] Finished loading data'.format(time.time() - start_time))

[25.6898591518] Finished loading data


In [6]:
train = dataPreProcessTime(train)
test = dataPreProcessTime(test)

y = train['is_attributed']
train.drop(['is_attributed', 'attributed_time'], axis=1, inplace=True)

In [7]:
sub = pd.DataFrame()
sub['click_id'] = test['click_id']
test.drop('click_id', axis=1, inplace=True)

train.drop('click_time', axis=1, inplace=True)
test.drop('click_time', axis=1, inplace=True)

print('[{}] Start XGBoost Training'.format(time.time() - start_time))

[83.6423830986] Start XGBoost Training


In [8]:
params = {'eta': 0.1, 
          'max_depth': 4, 
          'subsample': 0.9, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':100,
          'alpha':4,
          'objective': 'binary:logistic', 
          'eval_metric': 'auc', 
          'random_state': 99, 
          'scale_pos_weight': 150,
          'silent': True}
          
#x1, x2, y1, y2 = train_test_split(train, y, test_size=0.1, random_state=99)

In [9]:
from sklearn.model_selection import KFold

In [10]:
num_folds = 3
seed = 7 
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

In [22]:
train_x = train
for i,j in kf.split(train_x):
    train_x.ix[i]

In [11]:
train_x = train
train_y = y
for train_index, test_index in kf.split(train_x):
    watchlist = [(xgb.DMatrix(train_x.ix[train_index], train_y.ix[train_index]), 'train'), (xgb.DMatrix(train_x.ix[test_index], train_y.ix[test_index]), 'valid')]
    model = xgb.train(params, xgb.DMatrix(train_x.ix[train_index], train_y.ix[train_index]), 30 , watchlist, maximize=True, verbose_eval=10)
    print('[{}] Finish XGBoost Training'.format(time.time() - start_time))
    

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


[0]	train-auc:0.923313	valid-auc:0.922067
[10]	train-auc:0.920969	valid-auc:0.919468
[20]	train-auc:0.930514	valid-auc:0.929248
[29]	train-auc:0.936575	valid-auc:0.935607
[249.991210222] Finish XGBoost Training
[0]	train-auc:0.923415	valid-auc:0.921728
[10]	train-auc:0.929849	valid-auc:0.928341
[20]	train-auc:0.937852	valid-auc:0.936874
[29]	train-auc:0.941635	valid-auc:0.940244
[388.220332146] Finish XGBoost Training
[0]	train-auc:0.921895	valid-auc:0.923671
[10]	train-auc:0.927605	valid-auc:0.929733
[20]	train-auc:0.936074	valid-auc:0.937216
[29]	train-auc:0.940851	valid-auc:0.941946
[520.829543114] Finish XGBoost Training


In [31]:
#result  = cross_val_score(model, train, y , cv= kfold)
train_x = train.values
train_y = y.values 
for train_index, test_index in kf.split(train_x):
    xgb_model = xgb.XGBClassifier().fit(train_x[train_index], train_y[train_index])
    predictions = xgb_model.predict(train_x[test_index])
    actuals = train_y[test_index]
    print(confusion_matrix(actuals, predictions))

NameError: name 'confusion_matrix' is not defined

In [9]:
watchlist = [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
model = xgb.train(params, xgb.DMatrix(x1, y1), 800 , watchlist, maximize=True, verbose_eval=10)

print('[{}] Finish XGBoost Training'.format(time.time() - start_time))

[0]	train-auc:0.935956	valid-auc:0.931926
[10]	train-auc:0.952274	valid-auc:0.949015
[20]	train-auc:0.955979	valid-auc:0.952873
[30]	train-auc:0.958867	valid-auc:0.955856
[40]	train-auc:0.960911	valid-auc:0.958138
[50]	train-auc:0.962945	valid-auc:0.960157
[60]	train-auc:0.96469	valid-auc:0.961871
[70]	train-auc:0.965989	valid-auc:0.9631
[80]	train-auc:0.96717	valid-auc:0.964387
[90]	train-auc:0.967747	valid-auc:0.964887
[100]	train-auc:0.968272	valid-auc:0.965359
[110]	train-auc:0.968661	valid-auc:0.96567
[120]	train-auc:0.96904	valid-auc:0.965953
[130]	train-auc:0.969414	valid-auc:0.966223
[140]	train-auc:0.969698	valid-auc:0.966408
[150]	train-auc:0.970002	valid-auc:0.966595
[160]	train-auc:0.97026	valid-auc:0.966807
[170]	train-auc:0.970518	valid-auc:0.967064
[180]	train-auc:0.970793	valid-auc:0.967311
[190]	train-auc:0.970992	valid-auc:0.96748
[200]	train-auc:0.971155	valid-auc:0.967647
[210]	train-auc:0.971335	valid-auc:0.967778
[220]	train-auc:0.971536	valid-auc:0.967898
[230]	t

[1860]	train-auc:0.978888	valid-auc:0.970014
[1870]	train-auc:0.978908	valid-auc:0.970014
[1880]	train-auc:0.978921	valid-auc:0.970014
[1890]	train-auc:0.978937	valid-auc:0.970023
[1900]	train-auc:0.978954	valid-auc:0.970019
[1910]	train-auc:0.978971	valid-auc:0.970003
[1920]	train-auc:0.978989	valid-auc:0.969998
[1930]	train-auc:0.979008	valid-auc:0.97
[1940]	train-auc:0.97903	valid-auc:0.970006
[1950]	train-auc:0.979049	valid-auc:0.970002
[1960]	train-auc:0.97907	valid-auc:0.969991
[1970]	train-auc:0.979088	valid-auc:0.969978
[1980]	train-auc:0.979101	valid-auc:0.969966
[1990]	train-auc:0.979124	valid-auc:0.969967
[1999]	train-auc:0.97914	valid-auc:0.969966
[10788.5003619] Finish XGBoost Training


In [10]:
sub['is_attributed'] = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
sub.to_csv('xgb_sub.csv',index=False)